In [2]:
sys.path.append('C:/Users/nilso/Documents/EPFL/PDM/PDM_PINN/SciANN/DNN_TEST/sys/')

from loss import *
from unet import UNet
from dataloader import *

import torch.optim as optim

from BaseModel import BaseModel

In [3]:
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

In [10]:
training_data = dataset('../Training_Data/Moseley_Homogeneous/','../Training_Data/Moseley_Homogeneous_Event0000_Continuous.csv',event='Event0000')

#training_data = dataset('Moseley_Homogeneous_Big/','Simple_Homogeneous_Moseley_Event0000_Continuous.csv')

In [11]:
# # # Training
train_loader = DataLoader(training_data, batch_size=2, shuffle=True)

net = UNet(in_channels=4,out_channels=1)

optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=1e-6)

loss_fn = PINNLoss_MSE(device = 'cpu')

N = 3 # Number of time-steps to unroll

In [16]:
input = training_data.__getitem__(0)['wave_input'].transpose(1,0)[None]

In [18]:
input

torch.Size([1, 4, 300, 300])

In [13]:
input.dtype

torch.float32

In [14]:
# for epoch ... 

for i in range(0,3):

    print(i)
    loss_ar = 0
    optimizer.zero_grad()

    for j in range(0,N):
        
        output = net(input) # 1 x 4 x 300 x 300  --> 1 x 1 x 300 x 300

        pinn = torch.cat((input,output),axis=1) # 1 x 5 x 300 x 300 

        loss_ar = loss_ar + loss_fn(pinn)
        print(loss_ar)
        print(input.dtype)
        # update input data 
        input = pinn[:,1:,:,:] # 1 x 4 x 300 x 300 

        print(j)
    
    loss_ar.backward()
    optimizer.step()
    input = input.detach()


0
tensor(28.1282, grad_fn=<AddBackward0>)
torch.float32
0
tensor(55.2364, grad_fn=<AddBackward0>)
torch.float32
1
tensor(55.8814, grad_fn=<AddBackward0>)
torch.float32
2
1
tensor(0.5829, grad_fn=<AddBackward0>)
torch.float32
0
tensor(1.1763, grad_fn=<AddBackward0>)
torch.float32
1
tensor(2.2612, grad_fn=<AddBackward0>)
torch.float32
2
2
tensor(0.9365, grad_fn=<AddBackward0>)
torch.float32
0
tensor(1.8349, grad_fn=<AddBackward0>)
torch.float32
1
tensor(2.8345, grad_fn=<AddBackward0>)
torch.float32
2
